<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/fixed_income/term_structure_movement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from pandas_datareader import DataReader as pdr

In [2]:
files = ["DGS" + x for x in ["1", "2", "3", "5", "10", "20", "30"]]
df = None

# reading market yield on U.S. treasury securities data from FRED
for f in files:
    d = pdr(f, "fred", start=1920)
    df = pd.concat((df, d), axis=1)
df *= 100

# move index to column
df.index.name = "date"
df = df.reset_index()

# convert to monthly format and then save first day of the month
df["month"] = df.date.dt.to_period("M").astype(str)
df = df.groupby("month").first()
df = df.drop(columns=["date"])

# computing the monthly changes in yields (basis points) for each maturity securities
df = df.diff()
df = df.dropna(subset=["DGS30"])

# the monthly changes in yields (basis points)
columns = ["1-Year", "2-Year", "3-Year", "5-Year", "10-Year", "20-Year", "30-Year"]
df.columns = columns
df

,1-Year,2-Year,3-Year,5-Year,10-Year,20-Year,30-Year
month,,,,,,,
1977-03,7.0,-4.0,-6.0,8.0,8.0,13.0,14.0
1977-04,-20.0,-14.0,-12.0,-7.0,-6.0,-5.0,-6.0
1977-05,28.0,13.0,6.0,2.0,2.0,2.0,2.0
1977-06,15.0,9.0,0.0,-7.0,-7.0,-6.0,-5.0
1977-07,-13.0,-12.0,-13.0,-16.0,-4.0,-10.0,-17.0
...,...,...,...,...,...,...,...
2022-02,38.0,40.0,35.0,26.0,18.0,14.0,11.0
2022-03,13.0,13.0,8.0,-7.0,-9.0,0.0,-1.0
2022-04,81.0,113.0,114.0,99.0,67.0,41.0,33.0


In [3]:
# example
start = '1990-01'
stop = '2021-12'
df_1 = df.loc[start:stop]

# mean and std dev of the monthly changes in yields
tbl1 = df_1.describe().round(2).iloc[1:3]
tbl1 = tbl1.rename(index={"std": "std dev"}).reset_index()
tbl1.columns = [
    "Statistics",
    "1-Year",
    "2-Year",
    "3-Year",
    "5-Year",
    "10-Year",
    "20-Year",
    "30-Year",
]
tbl1

,Statistics,1-Year,2-Year,3-Year,5-Year,10-Year,20-Year,30-Year
0,mean,-1.93,-1.86,-1.78,-1.71,-1.66,-1.27,-1.59
1,std dev,21.84,25.32,26.82,27.56,26.39,24.15,23.04


In [4]:
# correlation matrix of the monthly changes in yields
df_1.corr().applymap("{0:.02f}".format)

,1-Year,2-Year,3-Year,5-Year,10-Year,20-Year,30-Year
1-Year,1.00,0.91,0.86,0.76,0.63,0.50,0.50
2-Year,0.91,1.00,0.98,0.92,0.79,0.66,0.63
3-Year,0.86,0.98,1.00,0.96,0.86,0.73,0.70
5-Year,0.76,0.92,0.96,1.00,0.95,0.85,0.81
10-Year,0.63,0.79,0.86,0.95,1.00,0.96,0.93
20-Year,0.50,0.66,0.73,0.85,0.96,1.00,0.98
30-Year,0.50,0.63,0.70,0.81,0.93,0.98,1.00
